<h2>General Channel statistics<h2>

## To customize depending on the user:
path = "C:\\Users\\john.doe\\Documents\\My_Directory"

My_API_KEY = 'my_api_key' # Credential requested via the Google developper platform 

In [2]:
# Import the libraries

import os
import csv
import requests
import itertools
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

from ggplot import *
import wes

from yt_stats import YTstats

from collections import Counter

from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

<h3>General statistics on the Channel and videos<h3>

In [3]:
# Read the stats ans videos
channel_id = 'my_channel_id'                
API_KEY = My_API_KEY    
yt = YTstats(My_API_KEY, channel_id)
yt.extract_all()
yt.dump() 
yt.channel_statistics
yt.video_data

NameError: name 'My_API_KEY' is not defined

In [ ]:
# Statistics
yt.channel_statistics

In [ ]:
# Extract the information ...
data_items = yt.video_data.items()
data_list = list(data_items)

In [ ]:
# ... in a dataframe
df = pd.DataFrame(data_items)
df = df[1].apply(pd.Series)
df.head(5)

In [ ]:
# Restrict to January
df = df.loc[df['publishedAt'].str.contains('2022-01-')]
df[['publishedAt','title','commentCount','categoryId', 'tags']].head(5)

In [ ]:
df.to_csv(path + 'VideosJanuary2022Details.csv', mode='w', index=True, sep=';', header=True)

In [ ]:
#Number of videos
len(df)

In [ ]:
# Search for the most commented video
df['commentCount'] = pd.to_numeric(df['commentCount'])
maxCount = max(df['commentCount'])
maxCount

In [ ]:
# Sort by number of comments
df = df.sort_values('commentCount', ascending=False)
df[['publishedAt','title','commentCount','categoryId', 'tags']].head(10)

In [ ]:
# Search for the most viewed video
df['viewCount'] = pd.to_numeric(df['viewCount'])
maxCount = max(df['viewCount'])
maxCount

In [ ]:
df = df.sort_values('viewCount', ascending=False)
df[['publishedAt','title','viewCount','categoryId', 'tags']].head(5)

<h3>Commented versus Liked diagram<h3>

In [ ]:
df = pd.read_csv(path + 'VideosJanuary2022Details.csv', sep=";")

In [ ]:
d = {'Comments': df['commentCount'], 'Views': df['viewCount'], 'Likes': df['likeCount'], 'Category': df['categoryId'], 'Tags': df['tags']}
Stats = pd.DataFrame(data=d)

Stats['Category'] = Stats['Category'].astype(str)
Stats['Tags'] = Stats['Tags'].astype(str)
Stats['Tags'] = Stats['Tags'].map(lambda x: x.replace('[',''))
Stats['Tags'] = Stats['Tags'].map(lambda x: x.replace(']',''))
Stats.head(5)

In [ ]:
cat = np.unique(Stats['Category'])
cat

In [ ]:
%%capture --no-display

ggplot(aes(x = 'Comments', y = 'Likes', size='Views', color='Category'), data = Stats) + \
    geom_point(alpha = 1) + \
    scale_x_log() + \
    scale_y_log() 

<h3> Tags word cloud<h3>

In [ ]:
Tags = Stats['Tags']
Tags = Tags.tolist()
Tags = list(itertools.chain.from_iterable([x.replace("'","").split(',') for x in Tags]))
Tags = list([x.lstrip() for x in Tags])
Tags = [x for x in Tags if str(x) != 'nan']
Tags[0:10]

In [ ]:
#!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

lemmatizer = FrenchLefffLemmatizer()
Tags_lemm = []
for t_ in Tags:
    Tags_lemm.append(lemmatizer.lemmatize(str(t_)))
Tags_lemm[0:10]    

In [ ]:
Tags_lemm_filtered = [word for word in Tags_lemm if word not in fr_stop]

In [ ]:
wordcloud = WordCloud(background_color = 'white', max_font_size=50).generate(str(Tags_lemm))
plt.figure()
plt.imshow(wordcloud)#, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
Counter(Tags_lemm_filtered).most_common()[0:15]

<h3> Save the YouTube video IDs <h3>

In [ ]:
import json
with open(path + 'my_file.json', 'r') as f:
  data = json.load(f)

In [ ]:
templist = data[channel_id]['video_data']
tempdf = pd.DataFrame.from_dict(templist, orient='index')
tempdf = tempdf.loc[tempdf['publishedAt'].str.contains('2022-01-')]
videoIds = list(tempdf.index)
videoIds[0:5]

In [ ]:
df = pd.DataFrame (videoIds, columns = ['videoIds'])
df.to_csv(path + 'VideoJanuary2022Ids.csv', mode='w', index=False, sep=';', header=True)